# A Brainstorm before starting
Taken from: Julien Danjou's Hacker's Guide To Python


In [2]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [1]:
import sys

sys.builtin_module_names

('_ast',
 '_bisect',
 '_codecs',
 '_codecs_cn',
 '_codecs_hk',
 '_codecs_iso2022',
 '_codecs_jp',
 '_codecs_kr',
 '_codecs_tw',
 '_collections',
 '_csv',
 '_datetime',
 '_functools',
 '_heapq',
 '_imp',
 '_io',
 '_json',
 '_locale',
 '_lsprof',
 '_md5',
 '_multibytecodec',
 '_pickle',
 '_random',
 '_sha1',
 '_sha256',
 '_sha512',
 '_sre',
 '_string',
 '_struct',
 '_symtable',
 '_thread',
 '_warnings',
 '_weakref',
 '_winapi',
 'array',
 'atexit',
 'audioop',
 'binascii',
 'builtins',
 'cmath',
 'errno',
 'faulthandler',
 'gc',
 'itertools',
 'marshal',
 'math',
 'mmap',
 'msvcrt',
 'nt',
 'operator',
 'parser',
 'signal',
 'sys',
 'time',
 'winreg',
 'xxsubtype',
 'zipimport',
 'zlib')

In [3]:
from itertools import permutations

print(sum(1 for _ in permutations([1, 2, 3, 4, 5])))

120


"Whenever you are about to write our own function to handle a simple task, please **stop** and look through the standard library first."

"There’s a few of the standard modules that you should definitely know about:

- operator: provides functions implementing the basic Python operators which you can use instead of having t owrite your own lambda expressions
- os: provides access to basic OS functions. 
- random: provides functions for generating pseudo-random numbers. 
- re: provides regular expression functionalities.
- functools
- iterators
- uuid: allows to autogenerate UUIDs(Universall Unique Identifiers)."


## Fixtures

"In unit testing, **fixtures** represent components that are set up before a test, and cleaned up ater the test is done. It’s usually a good idea to build a special kind of component for them."

### Basics

In [ ]:
## Unittest setUp() and tearDown(). Running code before and after unittest

import unittest

class TestMe(unittest.TestCase): 
    def setUp(self): 
        self.list = [1, 2, 3]
    
    def test_length(self): 
        self.list.append(4) 
        self.assertEqual(len(self.list), 4)
    
    def test_has_one(self): 
        self.assertEqual(len(self.list), 3) 
        self.assertIn(1, self.list)

### fixtures module

"The fixtures module provides a few built-in fixtures, like `fixtures.Environment` Variable – useful for adding or changing a variable in `os.environ` that will be reset upon test exit." 

In [9]:
import fixtures # not part of the standard library
import os

class TestEnviron(fixtures.TestWithFixtures): # inherits from unittest TestCase
    def test_environ(self): 
        fixture = self.useFixture(fixtures.EnvironmentVariable("FOOBAR", "42")) 
        self.assertEqual(os.environ.get("FOOBAR"), "42")
    def test_environ_no_fixture(self): 
        self.assertEqual(os.environ.get("FOOBAR"), None)


### Mocking
"Mock objects are simulated objects that mimic the behaviour of real application objects, but in particular and controlled ways. These are especially useful in creating environments that describe precisely the conditions for which you would like to testcode"

In [11]:
from unittest import mock 

m = mock.Mock() 
m.some_method.return_value = 42 

print(m.some_method())

def print_hello():
    print("hello world!") 
    return 43 

m.some_method.side_effect = print_hello 

print(m.some_method())

m.some_method.call_count  # stats on method calling

42
hello world!
43


2

Example: Using `mock.patch` to test a set of behaviour

In [ ]:
import requests 
import unittest 
import mock

class WhereIsPythonError(Exception): 
    pass

def is_python_still_a_programming_language(): 
    try: 
        r = requests.get("http://python.org") 
    except IOError: 
        pass 
    else: 
        if r.status_code == 200: 
            return 'Python is a programming language' in r.content 
    raise WhereIsPythonError("Something bad happened")
        
def get_fake_get(status_code, content): 
    m = mock.Mock() 
    m.status_code = status_code 
    m.content = content 
    def fake_get(url): 
        return m 
    return fake_get

def raise_get(url): 
    raise IOError("Unable to fetch url %s" % url)
    
class TestPython(unittest.TestCase): 
    @mock.patch('requests.get', get_fake_get(200, 'Python is a programming language for sure')) 
    def test_python_is(self): 
        self.assertTrue(is_python_still_a_programming_language())
        
    @mock.patch('requests.get', get_fake_get( 200, 'Python is no more a programming language')) 
    def test_python_is_not(self): 
        self.assertFalse(is_python_still_a_programming_language())

    @mock.patch('requests.get', get_fake_get( 404, 'Whatever')) 
    def test_bad_status_code(self): 
        self.assertRaises(WhereIsPythonError, is_python_still_a_programming_language)

    @mock.patch('requests.get', raise_get) 
    def test_ioerror(self): 
        self.assertRaises(WhereIsPythonError, is_python_still_a_programming_language)


### Scenarios
"This package provides an easy way to run a class test against a different set of scenarios generated at run-time."
Same example as in Mocking, the test runs three times because three scenarios are defined:

In [ ]:
import mock 
import requests 
import testscenarios

class WhereIsPythonError(Exception): 
    pass

def is_python_still_a_programming_language(): 
    r = requests.get("http://python.org") 
    if r.status_code == 200: 
        return 'Python is a programming language' in r.content 
    raise WhereIsPythonError("Something bad happened")

def get_fake_get(status_code, content): 
    m = mock.Mock() 
    m.status_code = status_code
    m.content = content 
    def fake_get(url): 
        return m 
    return fake_get

class TestPythonErrorCode(testscenarios.TestWithScenarios): 
    scenarios = [ ('Not found', dict(status=404)), 
                  ('Client error', dict(status=400)), 
                  ('Server error', dict(status=500)), ]
    
    def test_python_status_code_handling(self): 
        with mock.patch('requests.get', get_fake_get( self.status, 'Python is a programming language for sure')): 
            self.assertRaises(WhereIsPythonError, is_python_still_a_programming_language)


### Profiling 

In [7]:
import cProfile
import time

def print_hello():
    def print_it():
        for _ in range(0, 5):
            time.sleep(0.2)
        print('here')
    print("hello world!") 
    print_it()
    return 43 


cProfile.run('print_hello()')

hello world!
here
         501 function calls (497 primitive calls) in 1.005 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.005    1.005 <ipython-input-7-04c863fb3a13>:4(print_hello)
        1    0.000    0.000    1.004    1.004 <ipython-input-7-04c863fb3a13>:5(print_it)
        1    0.000    0.000    1.005    1.005 <string>:1(<module>)
        4    0.000    0.000    0.002    0.001 __init__.py:274(dumps)
        1    0.000    0.000    0.000    0.000 attrsettr.py:35(__getattr__)
        1    0.000    0.000    0.000    0.000 cp1252.py:11(encode)
        4    0.000    0.000    0.000    0.000 encoder.py:120(__init__)
        4    0.000    0.000    0.002    0.000 encoder.py:248(encode)
        4    0.000    0.000    0.000    0.000 encoder.py:277(iterencode)
        4    0.000    0.000    0.000    0.000 encoder.py:380(_make_iterencode)
       26    0.000    0.000    0.000    0.000 encoder.py:41(e